In [12]:
import pandas as pd
import json

# Load the Oil results dataset
results_oil = pd.read_csv('../../../data/outputs/results_oil.csv')

# Function to prepare the data for CanvasJS
def prepare_canvasjs_data(df, metric, company_col='Company', quarter_col='Quarter'):
    companies = df[company_col].unique()
    data_points = []
    
    for company in companies:
        company_data = df[df[company_col] == company]
        data_series = {
            "type": "line",
            "name": company,
            "showInLegend": True,
            "dataPoints": [{"label": str(row[quarter_col]), "y": row[metric]} for _, row in company_data.iterrows()]
        }
        data_points.append(data_series)
    
    return data_points

# Prepare the Oil data for CanvasJS (for example, Percentage Change)
oil_data = prepare_canvasjs_data(results_oil, 'Percentage Change (%)')

# Convert to JSON for embedding
oil_json = json.dumps(oil_data)

# Create the CanvasJS HTML structure
oil_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Oil Sector Data</title>
    <script src="https://canvasjs.com/assets/script/canvasjs.min.js"></script>
</head>
<body>
    <h2>Oil Sector - Percentage Change</h2>
    <div id="oilChart" style="height: 370px; width: 100%;"></div>
    <script>
        window.onload = function () {{
            var oilChart = new CanvasJS.Chart("oilChart", {{
                animationEnabled: true,
                theme: "light2",
                title: {{
                    text: "Quarterly Percentage Change - Oil Sector"
                }},
                axisY: {{
                    title: "Percentage Change (%)"
                }},
                legend: {{
                    cursor: "pointer",
                    itemclick: function(e) {{
                        if (typeof(e.dataSeries.visible) === "undefined" || e.dataSeries.visible) {{
                            e.dataSeries.visible = false;
                        }} else {{
                            e.dataSeries.visible = true;
                        }}
                        e.chart.render();
                    }}
                }},
                data: {oil_json}
            }});
            oilChart.render();
        }}
    </script>
</body>
</html>
"""

# Save the Oil plot as an HTML file
with open('../../../static/plots/oil/oil_plot_canvasjs.html', 'w') as f:
    f.write(oil_html)
